<a href="https://colab.research.google.com/github/brianegge/garbage_bin/blob/master/garbage_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tensorflow Object Detection with custom dataset in Google Colab

Jupyter notebook providing steps to retrain a [ModelZoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) model with custom dataset.

It runs in [Google Colab](https://colab.research.google.com) using [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection).

**Requirements are only dataset images and annotations file.**

**Colab Runtime type: Python3, GPU enabled.**



#Create Dataset

I generated dataset annotations with [LabelImg](https://github.com/tzutalin/labelImg).




# Install required packages


In [ ]:
%tensorflow_version 1.x

Mount Google Drive

The cell will return the following and your needs to go to the link to retrieve the authorization code. Then you are good to go!

In [ ]:
from google.colab import drive
import os
if not os.path.exists('/content/gdrive'):
    drive.mount('/content/gdrive')

root_path = '/content/gdrive/My Drive/dev/garbage_bin'  #change dir to your project folder

!mkdir -p /content/datalab
!rsync -az "{root_path}/" /content/datalab


In [ ]:
!pip install tf_slim

import tensorflow as tf
print(tf.__version__)

import os
os.chdir('/content/')
!rm -fr models
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/brianegge/garbage_bin.git src

os.chdir('/content/models/research')
#!git checkout ae0a9409212d0072938fa60c9f85740bb89ced7e
!git checkout master
!git apply --verbose < /content/src/anchor.patch
#!cat object_detection/anchor_generators/multiple_grid_anchor_generator.py
!protoc object_detection/protos/*.proto --python_out=.

import sys
sys.path.append('/content/models/research/slim')
import matplotlib.pyplot as plt
import os
os.environ['PYTHONPATH'] += ':/content/models:/content/models/research/:/content/models/research/slim/:/content/models/research/object_detection/utils/:/content/models/research/object_detection'

%cd /content/models/research/
!python object_detection/builders/model_builder_test.py

# Download pretrained model

To use another model from [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) change MODEL var.

In [ ]:
%mkdir -p /content/datalab
%cd /content/datalab
!rm -fr pretrained_model

import os
import shutil
import glob
import urllib
import tarfile


MODEL = 'ssd_mobilenet_v1_coco'
MODEL_DATE = '2018_01_28'
MODEL_FILE = MODEL + '_' + MODEL_DATE + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/datalab/fine_tuned_model/frozen_inference_graph.pb'

if not (os.path.exists(MODEL_FILE)):
  urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL + '_' + MODEL_DATE, DEST_DIR)

**`Create train and test data sets`**


In [ ]:

os.chdir('/content/datalab')
print(sys.path)
!echo "Running prepare in $(pwd)"
%shell ../src/scripts/create-label-map.py < images/labels.txt > annotations/label_map.pbtxt
%shell python ../src/scripts/xml_to_csv.py -i images -o annotations/labels.csv
%shell python ../src/scripts/split_labels.py
%shell PYTHONPATH=/content/models/research:/content/models/research/slim python ../src/scripts/generate_tfrecord.py --csv_input=annotations/train_labels.csv --label_map=annotations/label_map.pbtxt --img_path=images --output_path=annotations/train.record
%shell PYTHONPATH=/content/models/research:/content/models/research/slim python  ../src/scripts/generate_tfrecord.py --csv_input=annotations/test_labels.csv --label_map=annotations/label_map.pbtxt --img_path=images --output_path=annotations/test.record

# Edit model config file
To you use a different pretrained model in step before, update accordingly filename var and re.sub functions in next cell.


In [ ]:
%cd /content/datalab

import re

in_file = '/content/models/research/object_detection/samples/configs/{}.config'.format(MODEL)
pipeline_config_path = '/content/datalab/pretrained_model/pipeline.config'

# todo grab number of classes from /root/datalab/annotations/label_map.pbtxt

with open(in_file) as f:
  s = f.read()
with open(pipeline_config_path, 'w') as f:
  s = re.sub('num_classes: [0-9]+', 'num_classes: 4', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/datalab/pretrained_model/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/.*_train.record-\?\?\?\?\?-of-[0-9]*', '/content/datalab/annotations/train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record', '/content/datalab/annotations/train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/.*_val.record-\?\?\?\?\?-of-[0-9]*', '/content/datalab/annotations/test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record', '/content/datalab/annotations/test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/.*_map.pbtxt', '/content/datalab/annotations/label_map.pbtxt', s)
  s = re.sub('max_detections_per_class: 100', 'max_detections_per_class: 1', s)
  s = re.sub('random_horizontal_flip', 'random_adjust_brightness', s)
  s = re.sub('ssd_random_crop', 'random_adjust_saturation', s)
  s = re.sub('max_evals: 10', 'max_evals: 100', s)
  s = re.sub('batch_size: 24', 'batch_size: 8', s)
  s = re.sub('num_steps: 200000', 'num_steps: 500', s)
  
  #s = re.sub('keep_aspect_ratio_resizer','fixed_shape_resizer',s)
  #s = re.sub('min_dimension: 600','height: 960',s)
  #s = re.sub('max_dimension: 1024','width: 540',s)
  f.write(s)
#!cat {pipeline_config_path}

In [ ]:
# tensorboard setup - this doesn't work right now

# borrowed from https://colab.research.google.com/drive/1bUiykt6QIZPU586TnlL-GNSE74b506uX?usp=sharing#scrollTo=fO4xnKenbfQK
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
#the logs that are created while training 
LOG_DIR = "/content/datalab/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Train model
Set num_train_steps and num_eval_steps values to change train and eval steps in training process.



In [ ]:
%cd /content/datalab
! [ -d trained ] && rm -fr trained/* || mkdir -p trained

#!PYTHONPATH=.:/content/models:/content/models/research:/content/models/research/slim \
#python /content/models/research/object_detection/model_main.py \
#    --pipeline_config_path={pipeline_config_path} \
#    --model_dir=trained \
#    --alsologtostderr \
#    --num_train_steps=300

# Begin training
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!python /content/models/research/object_detection/legacy/train.py \
    --logtostderr \
    --train_dir=/content/datalab/trained \
    --pipeline_config_path={pipeline_config_path}

# export model after training
#!rm -fr fine_tuned_model/*
#lst = os.listdir('trained')
#lf = filter(lambda k: 'model.ckpt-' in k, lst)
#last_model = sorted(lf)[-1].replace('.meta', '')

#!PYTHONPATH=.:/content/models/research:/content/models/research/slim \
#python /content/models/research/object_detection/export_inference_graph.py \
#    --input_type=image_tensor \
#    --pipeline_config_path={pipeline_config_path} \
#    --output_directory=fine_tuned_model \
#    --trained_checkpoint_prefix=trained/$last_model

#!cp -rv /content/datalab/fine_tuned_model/frozen_inference_graph.pb "{root_path}/"   


#Export trained model

Export trained model with highest step number in filename.

In [ ]:
#!python -c 'import tensorflow as tf; print(tf.__version__)'

%cd /content/datalab

!mkdir -p fine_tuned_model

!rm -fr fine_tuned_model/*
lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

#!ls /content/datalab/trained/{last_model}*

%cd /content/models/research
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/:/content/models/research/object_detection/utils/:/content/models/research/object_detection'
!python object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/datalab/trained/pipeline.config \
    --output_directory=/content/datalab/fine_tuned_model \
    --trained_checkpoint_prefix=/content/datalab/trained/{last_model}

!cp -rv /content/datalab/fine_tuned_model/frozen_inference_graph.pb "{root_path}/"



# Run inference


In [ ]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from pprint import pprint

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#print(tf.__version__)
#if tf.__version__ < '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
  
# This is needed to display the images.
%matplotlib inline

from utils import label_map_util
from utils import visualization_utils as vis_util

# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/datalab/fine_tuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/datalab/annotations/', 'label_map.pbtxt')

NUM_CLASSES = 4

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/content/datalab/images'
TEST_IMAGE_PATHS = glob.glob(PATH_TO_TEST_IMAGES_DIR + '/*.jpg') + glob.glob(PATH_TO_TEST_IMAGES_DIR + '/*.png')
# Size, in inches, of the output images.
IMAGE_SIZE = (18, 12)

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


# last time images by time
for image_path in sorted(TEST_IMAGE_PATHS, key=os.path.getmtime)[-10:]:
  image = Image.open(image_path)
  image.resize((960, 540))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  #pprint(output_dict['detection_scores'])
  for index, score in enumerate(output_dict['detection_scores']):
    if score < 0.75:
      continue
    label = category_index[output_dict['detection_classes'][index]]['name']
    #print('{} has {}'.format(image_path,label))
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  

#Upload jpg image for inference

In [ ]:
%cd /content/datalab

!ls -ltrh /content/datalab/fine_tuned_model/

from google.colab import files
from os import path

uploaded = files.upload()
  
for name, data in uploaded.items():
  with open('image1.jpg', 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)